In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [76]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
note_mapping = {
    'C0': 1, 'C#0': 2, 'D0': 3, 'D#0': 4, 'E0': 5, 'F0': 6, 'F#0': 7, 'G0': 8, 'G#0': 9, 'A1': 10,
    'A#1': 11, 'B1': 12, 'C1': 13, 'C#1': 14, 'D1': 15, 'D#1': 16, 'E1': 17, 'F1': 18, 'F#1': 19,
    'G1': 20, 'G#1': 21, 'A2': 22, 'A#2': 23, 'B2': 24, 'C2': 25, 'C#2': 26, 'D2': 27, 'D#2': 28,
    'E2': 29, 'F2': 30, 'F#2': 31, 'G2': 32, 'G#2': 33, 'A3': 34, 'A#3': 35, 'B3': 36, 'C3': 37,
    'C#3': 38, 'D3': 39, 'D#3': 40, 'E3': 41, 'F3': 42, 'F#3': 43, 'G3': 44, 'G#3': 45, 'A4': 46,
    'A#4': 47, 'B4': 48, 'C4': 49, 'C#4': 50, 'D4': 51, 'D#4': 52, 'E4': 53, 'F4': 54, 'F#4': 55,
    'G4': 56, 'G#4': 57, 'A5': 58, 'A#5': 59, 'B5': 60, 'C5': 61, 'C#5': 62, 'D5': 63, 'D#5': 64,
    'E5': 65, 'F5': 66, 'F#5': 67, 'G5': 68, 'G#5': 69, 'A6': 70, 'A#6': 71, 'B6': 72, 'C6': 73
}

In [36]:
dataset = [
    (['C1', 'G1', 'A1', 'F1'], (['C1', 'E1', 'G1'], ['G1', 'B2', 'D2'], ['A1', 'C1', 'E1'], ['F1', 'A2', 'C2'])),
    (['C4', 'F4', 'G4', 'A5'], (['C4', 'E4', 'G4'], ['F4', 'C5', 'A5'], ['G4', 'B5', 'D5'], ['A5', 'C5', 'A6'])),
    (['C2', 'D2', 'F2', 'G2'], (['C2', 'G2', 'E3'], ['D2', 'A3', 'F2', 'D3'], ['F2', 'A3', 'C3'], ['G2', 'B3', 'D3']))
]

In [ ]:
data_frame = pd.read_csv('output.csv')

In [64]:
def encode_notes(notes):
    return [note_mapping[note] for note in notes]

In [83]:
# Process the dataset
X_train = []
y_train = []

for base_notes, chord_progressions in dataset:
    # Encode base notes
    encoded_base_notes = encode_notes(base_notes)

    print(base_notes)
    print(chord_progressions)

    # Encode and pad chord progressions
    encoded_chord_progressions = [encode_notes(chord) for chord in chord_progressions]
    padded_chord_progressions = pad_sequences(encoded_chord_progressions, maxlen=6, padding='post')

    # Create input-output pairs
    X_train.append(encoded_base_notes)
    y_train.append(padded_chord_progressions)

X_train = np.array(X_train)
y_train = np.array(y_train)

print("Here's X:")
print(X_train)
print("Here's y:")
print(y_train)

['C1', 'G1', 'A1', 'F1']
(['C1', 'E1', 'G1'], ['G1', 'B2', 'D2'], ['A1', 'C1', 'E1'], ['F1', 'A2', 'C2'])
['C4', 'F4', 'G4', 'A5']
(['C4', 'E4', 'G4'], ['F4', 'C5', 'A5'], ['G4', 'B5', 'D5'], ['A5', 'C5', 'A6'])
['C2', 'D2', 'F2', 'G2']
(['C2', 'G2', 'E3'], ['D2', 'A3', 'F2', 'D3'], ['F2', 'A3', 'C3'], ['G2', 'B3', 'D3'])
Here's X:
[[13 20 10 18]
 [49 54 56 58]
 [25 27 30 32]]
Here's y:
[[[13 17 20  0  0  0]
  [20 24 27  0  0  0]
  [10 13 17  0  0  0]
  [18 22 25  0  0  0]]

 [[49 53 56  0  0  0]
  [54 61 58  0  0  0]
  [56 60 63  0  0  0]
  [58 61 70  0  0  0]]

 [[25 32 41  0  0  0]
  [27 34 30 39  0  0]
  [30 34 37  0  0  0]
  [32 36 39  0  0  0]]]


In [87]:
# model = Sequential()
# model.add(Dense(units=128, activation = "relu"))
# model.add(Dense(units=4, activation='softmax'))

model = Sequential()
model.add(Dense(units=128, activation="relu", input_shape=(4,)))
model.add(RepeatVector(4))  # Repeat the vector to match the number of timesteps
model.add(LSTM(units=64, return_sequences=True))
model.add(TimeDistributed(Dense(units=6, activation='softmax')))

In [88]:
model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), loss='categorical_crossentropy', metrics=['accuracy'])

In [89]:
model.fit(X_train, y_train, epochs=50, batch_size=1, validation_split=0.2)

Epoch 1/50
2/2 [==============================] - 2s 531ms/step - loss: 207.0305 - accuracy: 0.3750 - val_loss: 165.2913 - val_accuracy: 0.7500
Epoch 2/50
2/2 [==============================] - 0s 43ms/step - loss: 172.3259 - accuracy: 0.8750 - val_loss: 157.3704 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 0s 39ms/step - loss: 153.8839 - accuracy: 0.6250 - val_loss: 157.4447 - val_accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 0s 45ms/step - loss: 147.9691 - accuracy: 0.1250 - val_loss: 157.3396 - val_accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 0s 39ms/step - loss: 144.4823 - accuracy: 0.2500 - val_loss: 157.6780 - val_accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 0s 38ms/step - loss: 140.9260 - accuracy: 0.6250 - val_loss: 156.9081 - val_accuracy: 0.7500
Epoch 7/50
2/2 [==============================] - 0s 33ms/step - loss: 137.0663 - accuracy: 0.8750 - val_loss: 157.2000 - val

In [91]:
print(np.array([13, 20, 10, 18]))

[13 20 10 18]


In [92]:
model.predict(np.array([[13, 20, 10, 18]]))

1/1 [==============================] - 0s 405ms/step


array([[[0.26685268, 0.33614048, 0.38286728, 0.00513131, 0.0058268 ,
         0.00318144],
        [0.26902235, 0.33629292, 0.39034677, 0.00158405, 0.00194457,
         0.00080937],
        [0.27066034, 0.33491242, 0.3908969 , 0.00127922, 0.00163507,
         0.00061604],
        [0.27201894, 0.33460954, 0.38996762, 0.00122181, 0.00160547,
         0.00057662]]], dtype=float32)